In [1]:
import lovely_tensors as lt
from lovely_numpy import lovely, set_config
from rich import print

lt.monkey_patch()
set_config(repr=lovely)
print("Loaded")

Loaded

In [2]:
from ll.actsave import ActivationLoader

escn = ActivationLoader.from_latest_version("/mnt/shared/ocp-actsave/escn/")
# print(escn)

rhescn = ActivationLoader.from_latest_version("/mnt/shared/ocp-actsave/rhescnv8clean/")
# print(rhescn)


def toplevel_keys(activations):
    return [k for k in activations.keys() if k.count(".") <= 1]


print(
    {
        "escn": toplevel_keys(escn.activations),
        "rhescn": toplevel_keys(rhescn.activations),
    }
)

{
    'escn': [
        'wigner',
        'wigner_inv',
        'x_embedding_l0m0',
        'x_embedding',
        'LayerBlock_0.x',
        'LayerBlock_0.x_message',
        'LayerBlock_0.x_message_updated',
        'LayerBlock_0.x_updated',
        'LayerBlock_1.x',
        'LayerBlock_1.x_message',
        'LayerBlock_1.x_message_updated',
        'LayerBlock_2.x',
        'LayerBlock_1.x_updated',
        'LayerBlock_2.x_message',
        'LayerBlock_2.x_message_updated',
        'LayerBlock_2.x_updated',
        'LayerBlock_3.x',
        'LayerBlock_3.x_updated',
        'LayerBlock_3.x_message_updated',
        'LayerBlock_3.x_message',
        'LayerBlock_4.x',
        'LayerBlock_4.x_message_updated',
        'LayerBlock_4.x_message',
        'LayerBlock_4.x_updated',
        'LayerBlock_5.x',
        'LayerBlock_5.x_message',
        'LayerBlock_5.x_message_updated',
        'LayerBlock_5.x_updated',
        'LayerBlock_6.x',
        'LayerBlock_6.x_message',
        'LayerBlock_6.x_message_updated',
        'LayerBlock_6.x_updated',
        'LayerBlock_7.x',
        'LayerBlock_7.x_message',
        'LayerBlock_7.x_message_updated',
        'LayerBlock_7.x_updated',
        'LayerBlock_8.x',
        'LayerBlock_8.x_message_updated',
        'LayerBlock_8.x_message',
        'LayerBlock_8.x_updated',
        'LayerBlock_9.x',
        'LayerBlock_9.x_message_updated',
        'LayerBlock_9.x_updated',
        'LayerBlock_9.x_message',
        'LayerBlock_10.x',
        'LayerBlock_10.x_message_updated',
        'LayerBlock_10.x_message',
        'LayerBlock_10.x_updated',
        'LayerBlock_11.x',
        'LayerBlock_11.x_message',
        'LayerBlock_11.x_message_updated',
        'x_pt',
        'LayerBlock_11.x_updated',
        'node_energy',
        'forces',
        'energy'
    ],
    'rhescn': [
        'full_wigner',
        'wigner_tri_to_rh',
        'wigner_inv_from_grid',
        'M0L0Embedding.x_l0m0',
        'M0L0Embedding.x',
        'LayerBlock_0.x',
        'LayerBlock_0.x_message',
        'LayerBlock_1.x',
        'LayerBlock_1.x_message',
        'LayerBlock_2.x',
        'LayerBlock_2.x_message',
        'LayerBlock_3.x',
        'LayerBlock_3.x_message',
        'LayerBlock_4.x',
        'LayerBlock_4.x_message',
        'LayerBlock_5.x',
        'LayerBlock_5.x_message',
        'LayerBlock_6.x',
        'LayerBlock_6.x_message',
        'LayerBlock_7.x',
        'LayerBlock_7.x_message',
        'LayerBlock_8.x',
        'LayerBlock_8.x_message',
        'LayerBlock_9.x',
        'LayerBlock_9.x_message',
        'LayerBlock_10.x',
        'LayerBlock_10.x_message',
        'LayerBlock_11.x',
        'LayerBlock_11.x_message',
        'x_pt',
        'node_energy',
        'forces',
        'energy'
    ]
}

In [3]:
import numpy as np
from bidict import bidict
from einops import rearrange
from IPython.display import Markdown, display

LMAX = 4
MMAX = 2
RH_MMAX = 4


def trimask(x: np.ndarray, lmax: int = LMAX):
    return x[:, : (lmax + 1) ** 2]


def _escnindices(lmax: int, mmax: int | None):
    coeffs = bidict[tuple[int, int], int]()
    for l in range(lmax + 1):
        for m in range(-l, l + 1):
            if mmax is not None and abs(m) > mmax:
                continue

            coeffs[(l, m)] = len(coeffs)
    return coeffs


def trimask_mmax(
    x: np.ndarray,
    lmax: int = LMAX,
    mmax: int = MMAX,
):
    indices: list[int] = []
    i = 0
    for l in range(lmax + 1):
        for m in range(-l, l + 1):
            if abs(m) > mmax:
                i += 1
                continue

            indices.append(i)
            i += 1

    return x[:, indices]


def rhmask(
    x: np.ndarray,
    lmax: int = LMAX,
    mmax: int | None = MMAX,
    rh_mmax: int = RH_MMAX,
):
    if x.ndim == 3:
        x = rearrange(
            x,
            "E (m two_sign l) C -> E m two_sign l C",
            m=rh_mmax + 1,
            two_sign=2,
        )

    idx = _escnindices(lmax, mmax)
    x_masked = np.zeros((x.shape[0], len(idx), *x.shape[4:]), dtype=x.dtype)
    for (l, m), i in idx.items():
        signidx = int(np.signbit(m).item())
        x_masked[:, i] = x[:, abs(m), signidx, l - abs(m)]
    return x_masked

In [4]:
display(Markdown("## Wigner"))

x_rh = rhescn.activations["full_wigner"][0]
x_rh = trimask(x_rh).swapaxes(-1, -2)
x_rh = trimask(x_rh).swapaxes(-1, -2)


print(
    {
        "rhescn": x_rh,
        "escn": (x_escn := escn.activations["wigner"][0]),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Wigner

{
    'rhescn': array[3137, 25, 25] f32 n=1960625 (7.5Mb) x∈[-1.000, 1.000] μ=0.001 σ=0.200,
    'escn': array[3137, 25, 25] f32 n=1960625 (7.5Mb) x∈[-1.000, 1.000] μ=0.001 σ=0.200
}

In [5]:
display(Markdown("## Wigner"))

x_rh = rhescn.activations["wigner_tri_to_rh"][0]
x_rh = rearrange(
    x_rh,
    "E (m1 two1 l1) l_sq -> E m1 two1 l1 l_sq",
    m1=RH_MMAX + 1,
    two1=2,
)
x_rh = rhmask(x_rh, mmax=None).swapaxes(-1, -2)
x_rh = trimask(x_rh).swapaxes(-1, -2)


print(
    {
        "rhescn": x_rh,
        "escn": (x_escn := escn.activations["wigner"][0]),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Wigner

{
    'rhescn': array[3137, 25, 25] f32 n=1960625 (7.5Mb) x∈[-1.000, 1.000] μ=0.001 σ=0.200,
    'escn': array[3137, 25, 25] f32 n=1960625 (7.5Mb) x∈[-1.000, 1.000] μ=0.001 σ=0.200
}

In [6]:
display(Markdown("## Embedding Layer"))
print(
    {
        "rhescn": (x_rh := rhescn.activations["M0L0Embedding.x_l0m0"][0]),
        "escn": (x_escn := escn.activations["x_embedding_l0m0"][0]),
    },
)
np.testing.assert_allclose(x_rh, x_escn)

print(
    {
        "rhescn": (x_rh := trimask(rhescn.activations["M0L0Embedding.x"][0])),
        "escn": (x_escn := escn.activations["x_embedding"][0]),
    },
)
np.testing.assert_allclose(x_rh, x_escn)

## Embedding Layer

{
    'rhescn': array[149, 128] f32 n=19072 (74Kb) x∈[-0.887, 1.319] μ=0.038 σ=0.355,
    'escn': array[149, 128] f32 n=19072 (74Kb) x∈[-0.887, 1.319] μ=0.038 σ=0.355
}

{
    'rhescn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-0.887, 1.319] μ=0.002 σ=0.071,
    'escn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-0.887, 1.319] μ=0.002 σ=0.071
}

In [7]:
display(Markdown("## Layer 0 x_edge"))
print(
    {
        "rhescn": (x_rh := rhescn.activations["LayerBlock_0.MessageBlock.x_edge"][0]),
        "escn": (x_escn := escn.activations["LayerBlock_0.message_block.x_edge"][0]),
    },
)
np.testing.assert_allclose(x_rh, x_escn)

## Layer 0 x_edge

{
    'rhescn': array[3137, 128] f32 n=401536 (1.5Mb) x∈[-0.278, 4.574] μ=-0.009 σ=0.064,
    'escn': array[3137, 128] f32 n=401536 (1.5Mb) x∈[-0.278, 4.574] μ=-0.009 σ=0.064
}

In [8]:
display(Markdown("## Layer 0 x_source"))
print(
    {
        "rhescn": (
            x_rh := trimask(rhescn.activations["LayerBlock_0.MessageBlock.x_source"][0])
        ),
        "escn": (x_escn := escn.activations["LayerBlock_0.message_block.x_source"][0]),
    },
)
np.testing.assert_allclose(x_rh, x_escn)

display(Markdown("## Layer 0 x_target"))
print(
    {
        "rhescn": (
            x_rh := trimask(rhescn.activations["LayerBlock_0.MessageBlock.x_target"][0])
        ),
        "escn": (x_escn := escn.activations["LayerBlock_0.message_block.x_target"][0]),
    },
)
np.testing.assert_allclose(x_rh, x_escn)

## Layer 0 x_source

{
    'rhescn': array[3137, 25, 128] f32 n=10038400 (38Mb) x∈[-0.887, 1.319] μ=0.002 σ=0.071,
    'escn': array[3137, 25, 128] f32 n=10038400 (38Mb) x∈[-0.887, 1.319] μ=0.002 σ=0.071
}

## Layer 0 x_target

{
    'rhescn': array[3137, 25, 128] f32 n=10038400 (38Mb) x∈[-0.887, 1.319] μ=0.001 σ=0.071,
    'escn': array[3137, 25, 128] f32 n=10038400 (38Mb) x∈[-0.887, 1.319] μ=0.001 σ=0.071
}

In [9]:
display(Markdown("## Layer 0 x_source_rot"))
print(
    {
        "rhescn": (
            x_rh := rhmask(
                rhescn.activations["LayerBlock_0.MessageBlock.x_source_rot"][0]
            )
        ),
        "escn": (
            x_escn := escn.activations["LayerBlock_0.message_block.x_source_rot"][0]
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn)

display(Markdown("## Layer 0 x_target_rot"))
print(
    {
        "rhescn": (
            x_rh := rhmask(
                rhescn.activations["LayerBlock_0.MessageBlock.x_target_rot"][0]
            )
        ),
        "escn": (
            x_escn := escn.activations["LayerBlock_0.message_block.x_target_rot"][0]
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn)

## Layer 0 x_source_rot

{
    'rhescn': array[3137, 19, 128] f32 n=7629184 (29Mb) x∈[-0.887, 1.319] μ=0.002 σ=0.082,
    'escn': array[3137, 19, 128] f32 n=7629184 (29Mb) x∈[-0.887, 1.319] μ=0.002 σ=0.082
}

## Layer 0 x_target_rot

{
    'rhescn': array[3137, 19, 128] f32 n=7629184 (29Mb) x∈[-0.887, 1.319] μ=0.002 σ=0.082,
    'escn': array[3137, 19, 128] f32 n=7629184 (29Mb) x∈[-0.887, 1.319] μ=0.002 σ=0.082
}

In [10]:
display(Markdown("## Layer 0 x_source_updated"))
print(
    {
        "rhescn": (
            x_rh := rhmask(
                rhescn.activations[
                    "LayerBlock_0.MessageBlock.so2_block_source.x_source_updated"
                ][0]
            )
        ),
        "escn": (
            x_escn := escn.activations["LayerBlock_0.message_block.x_source_post_so2"][
                0
            ]
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn)

display(Markdown("## Layer 0 x_target_updated"))
print(
    {
        "rhescn": (
            x_rh := rhmask(
                rhescn.activations[
                    "LayerBlock_0.MessageBlock.so2_block_target.x_target_updated"
                ][0]
            )
        ),
        "escn": (
            x_escn := escn.activations["LayerBlock_0.message_block.x_target_post_so2"][
                0
            ]
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn)

## Layer 0 x_source_updated

{
    'rhescn': array[3137, 19, 128] f32 n=7629184 (29Mb) x∈[-7.068, 8.137] μ=-0.002 σ=0.084,
    'escn': array[3137, 19, 128] f32 n=7629184 (29Mb) x∈[-7.068, 8.137] μ=-0.002 σ=0.084
}

## Layer 0 x_target_updated

{
    'rhescn': array[3137, 19, 128] f32 n=7629184 (29Mb) x∈[-9.294, 6.760] μ=-0.001 σ=0.081,
    'escn': array[3137, 19, 128] f32 n=7629184 (29Mb) x∈[-9.294, 6.760] μ=-0.001 σ=0.081
}

In [11]:
display(Markdown("## Layer 0 x_updated"))
print(
    {
        "rhescn": (
            x_rh := rhmask(rhescn.activations["LayerBlock_0.MessageBlock.x_updated"][0])
        ),
        "escn": (x_escn := escn.activations["LayerBlock_0.message_block.x_updated"][0]),
    },
)
np.testing.assert_allclose(x_rh, x_escn)

## Layer 0 x_updated

{
    'rhescn': array[3137, 19, 128] f32 n=7629184 (29Mb) x∈[-9.775, 9.056] μ=-0.003 σ=0.113,
    'escn': array[3137, 19, 128] f32 n=7629184 (29Mb) x∈[-9.775, 9.056] μ=-0.003 σ=0.113
}

In [12]:
display(Markdown("## Layer 0 grid_act to_grid"))


print(
    {
        "rhescn": (
            x_rh := (
                rearrange(
                    rhescn.activations[
                        "LayerBlock_0.MessageBlock.act_rotate_inv.x_grid"
                    ][0],
                    "E (res_beta res_alpha) C -> E res_beta res_alpha C",
                    res_beta=10,
                )
            )
        ),
        "escn": (
            x_escn := (
                escn.activations["LayerBlock_0.message_block.grid_act.x_grid"][0]
            )
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0 grid_act to_grid

{
    'rhescn': array[3137, 10, 5, 128] f32 n=20076800 (77Mb) x∈[-14.374, 12.972] μ=-0.009 σ=0.194,
    'escn': array[3137, 10, 5, 128] f32 n=20076800 (77Mb) x∈[-14.374, 12.972] μ=-0.009 σ=0.194
}

In [13]:
display(Markdown("## Layer 0 grid_act silu"))


print(
    {
        "rhescn": (
            x_rh := (
                rearrange(
                    rhescn.activations[
                        "LayerBlock_0.MessageBlock.act_rotate_inv.x_grid_silu"
                    ][0],
                    "E (res_beta res_alpha) C -> E res_beta res_alpha C",
                    res_beta=10,
                )
            )
        ),
        "escn": (
            x_escn := (
                escn.activations["LayerBlock_0.message_block.grid_act.x_grid_silu"][0]
            )
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0 grid_act silu

{
    'rhescn': array[3137, 10, 5, 128] f32 n=20076800 (77Mb) x∈[-0.278, 12.972] μ=0.003 σ=0.117,
    'escn': array[3137, 10, 5, 128] f32 n=20076800 (77Mb) x∈[-0.278, 12.972] μ=0.003 σ=0.117
}

In [14]:
display(Markdown("## Layer 0 grid_act to_sphere"))


print(
    {
        "rhescn": (
            x_rh := trimask_mmax(
                rhescn.activations[
                    "LayerBlock_0.MessageBlock.act_rotate_inv.x_sphere_silu"
                ][0]
            )
        ),
        "escn": (
            x_escn := (
                escn.activations["LayerBlock_0.message_block.grid_act.x_sphere_silu"][0]
            )
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0 grid_act to_sphere

{
    'rhescn': array[3137, 19, 128] f32 n=7629184 (29Mb) x∈[-6.431, 9.333] μ=0.000 σ=0.067,
    'escn': array[3137, 19, 128] f32 n=7629184 (29Mb) x∈[-6.431, 9.333] μ=0.000 σ=0.067
}

In [15]:
display(Markdown("## Layer 0 rotate_inv"))


print(
    {
        "rhescn": (
            x_rh := trimask(
                rhescn.activations[
                    "LayerBlock_0.MessageBlock.act_rotate_inv.x_sphere_rotated"
                ][0]
            )
        ),
        "escn": (
            x_escn := (
                escn.activations["LayerBlock_0.message_block.x_target_rot_inv"][0]
            )
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0 rotate_inv

{
    'rhescn': array[3137, 25, 128] f32 n=10038400 (38Mb) x∈[-5.364, 9.333] μ=8.411e-05 σ=0.058,
    'escn': array[3137, 25, 128] f32 n=10038400 (38Mb) x∈[-5.364, 9.333] μ=8.411e-05 σ=0.058
}

In [16]:
display(Markdown("## Layer 0 scatter"))


print(
    {
        "rhescn": (
            x_rh := trimask(
                rhescn.activations["LayerBlock_0.MessageBlock.x_scattered"][0]
            )
        ),
        "escn": (
            x_escn := (
                escn.activations["LayerBlock_0.message_block.x_target_reduce"][0]
            )
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0 scatter

{
    'rhescn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-6.183, 8.206] μ=0.002 σ=0.245,
    'escn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-6.183, 8.206] μ=0.002 σ=0.245
}

In [17]:
display(Markdown("## Layer 0 x_message"))


print(
    {
        "rhescn": (x_rh := trimask(rhescn.activations["LayerBlock_0.x_message"][0])),
        "escn": (
            x_escn := (
                escn.activations["LayerBlock_0.message_block.x_target_reduce"][0]
            )
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0 x_message

{
    'rhescn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-6.183, 8.206] μ=0.002 σ=0.245,
    'escn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-6.183, 8.206] μ=0.002 σ=0.245
}

In [18]:
display(Markdown("## Layer 0 conv"))


print(
    {
        "rhescn": (
            x_rh := trimask(
                rhescn.activations[
                    "LayerBlock_0.pointwise_grid_conv.x_message_updated"
                ][0]
            )
        ),
        "escn": (x_escn := (escn.activations["LayerBlock_0.x_message_updated"][0])),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0 conv

{
    'rhescn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-7.226, 11.187] μ=-0.001 σ=0.235,
    'escn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-7.226, 11.187] μ=-0.001 σ=0.235
}

In [19]:
display(Markdown("## Layer 0 x_message after SO2"))


print(
    {
        "rhescn": (x_rh := trimask(rhescn.activations["LayerBlock_0.x_message"][0])),
        "escn": (x_escn := escn.activations["LayerBlock_0.x_message"][0]),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0 x_message after SO2

{
    'rhescn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-6.183, 8.206] μ=0.002 σ=0.245,
    'escn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-6.183, 8.206] μ=0.002 σ=0.245
}

In [20]:
display(Markdown("## Layer 0 conv: x_grid"))


print(
    {
        "rhescn": (
            x_rh := rhescn.activations[
                "LayerBlock_0.pointwise_grid_conv.s2_conv.x_grid"
            ][0]
        ),
        "escn": (
            x_escn := rearrange(
                escn.activations["LayerBlock_0.grid_conv.x_grid"][0],
                "N res_beta res_alpha C -> N (res_beta res_alpha) C",
            )
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0 conv: x_grid

{
    'rhescn': array[149, 110, 128] f32 n=2097920 (8.0Mb) x∈[-0.250, 0.372] μ=0.011 σ=0.100,
    'escn': array[149, 110, 128] f32 n=2097920 (8.0Mb) x∈[-0.250, 0.372] μ=0.011 σ=0.100
}

In [21]:
display(Markdown("## Layer 0 conv: x_grid_message"))


print(
    {
        "rhescn": (
            x_rh := rhescn.activations[
                "LayerBlock_0.pointwise_grid_conv.s2_conv.x_message_grid"
            ][0]
        ),
        "escn": (
            x_escn := rearrange(
                escn.activations["LayerBlock_0.grid_conv.x_grid_message"][0],
                "N res_beta res_alpha C -> N (res_beta res_alpha) C",
            )
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0 conv: x_grid_message

{
    'rhescn': array[149, 110, 128] f32 n=2097920 (8.0Mb) x∈[-4.396, 13.227] μ=0.018 σ=0.347,
    'escn': array[149, 110, 128] f32 n=2097920 (8.0Mb) x∈[-4.396, 13.227] μ=0.018 σ=0.347
}

In [22]:
display(Markdown("## Layer 0 conv: to_grid_mat"))


print(
    {
        "rhescn": (
            x_rh := trimask(
                rhescn.activations["LayerBlock_0.pointwise_grid_conv.to_grid_sh_tri"][0]
            )
        ),
        "escn": (
            x_escn := rearrange(
                escn.activations[
                    "LayerBlock_0.grid_conv.x_message_to_grid.to_grid_mat"
                ][0],
                "res_beta res_alpha l_sq -> (res_beta res_alpha) l_sq",
            )
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0 conv: to_grid_mat

{
    'rhescn': array[110, 25] f32 n=2750 (11Kb) x∈[-0.692, 0.745] μ=0.022 σ=0.281,
    'escn': array[110, 25] f32 n=2750 (11Kb) x∈[-0.692, 0.745] μ=0.022 σ=0.281
}

In [23]:
display(Markdown("## Layer 0 conv: x_grid_conv"))


print(
    {
        "rhescn": (
            x_rh := rhescn.activations[
                "LayerBlock_0.pointwise_grid_conv.s2_conv.x_grid_conv"
            ][0]
        ),
        "escn": (
            x_escn := rearrange(
                escn.activations["LayerBlock_0.grid_conv.x_grid_conv"][0],
                "N res_beta res_alpha C -> N (res_beta res_alpha) C",
            )
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0 conv: x_grid_conv

{
    'rhescn': array[149, 110, 128] f32 n=2097920 (8.0Mb) x∈[-4.639, 7.088] μ=-0.009 σ=0.332,
    'escn': array[149, 110, 128] f32 n=2097920 (8.0Mb) x∈[-4.639, 7.088] μ=-0.009 σ=0.332
}

In [24]:
display(Markdown("## Layer 0 conv: x_message_from_grid"))


print(
    {
        "rhescn": (
            x_rh := trimask(
                rhescn.activations[
                    "LayerBlock_0.pointwise_grid_conv.x_message_updated"
                ][0]
            )
        ),
        "escn": (
            x_escn := escn.activations["LayerBlock_0.grid_conv.x_message_from_grid"][0]
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0 conv: x_message_from_grid

{
    'rhescn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-7.226, 11.187] μ=-0.001 σ=0.235,
    'escn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-7.226, 11.187] μ=-0.001 σ=0.235
}

In [25]:
for layer_idx in range(12):
    display(Markdown(f"## Layer {layer_idx}: Full"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.pointwise_grid_conv.x_message_updated"
                    ][0]
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[f"LayerBlock_{layer_idx}.x_message_updated"][0]
                )
            ),
        },
    )
    # np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 0: Full

{
    'rhescn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-7.226, 11.187] μ=-0.001 σ=0.235,
    'escn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-7.226, 11.187] μ=-0.001 σ=0.235
}

## Layer 1: Full

{
    'rhescn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-2.837, 3.452] μ=0.002 σ=0.142,
    'escn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-2.837, 3.452] μ=0.002 σ=0.142
}

## Layer 2: Full

...

{
    'rhescn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-3.096, 3.453] μ=-0.001 σ=0.234,
    'escn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-3.096, 3.453] μ=-0.001 σ=0.234
}

## Layer 10: Full

{
    'rhescn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-3.106, 3.480] μ=0.001 σ=0.261,
    'escn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-3.106, 3.480] μ=0.001 σ=0.261
}

## Layer 11: Full

{
    'rhescn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-29.861, 25.351] μ=0.010 σ=0.807,
    'escn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-29.861, 25.351] μ=0.010 σ=0.807
}

In [28]:
def check_layer(layer_idx: int):
    display(Markdown(f"## Layer {layer_idx} x_edge"))
    print(
        {
            "rhescn": (
                x_rh := rhescn.activations[
                    f"LayerBlock_{layer_idx}.MessageBlock.x_edge"
                ][0]
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_edge"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)

    display(Markdown(f"## Layer {layer_idx} x_source"))
    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[f"LayerBlock_{layer_idx}.MessageBlock.x_source"][
                        0
                    ]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_source"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)

    display(Markdown(f"## Layer {layer_idx} x_target"))
    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[f"LayerBlock_{layer_idx}.MessageBlock.x_target"][
                        0
                    ]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_target"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)

    display(Markdown(f"## Layer {layer_idx} x_source_rot"))
    print(
        {
            "rhescn": (
                x_rh := rhmask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.x_source_rot"
                    ][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_source_rot"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)

    display(Markdown(f"## Layer {layer_idx} x_target_rot"))
    print(
        {
            "rhescn": (
                x_rh := rhmask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.x_target_rot"
                    ][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_target_rot"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)

    display(Markdown(f"## Layer {layer_idx} x_source_updated"))
    print(
        {
            "rhescn": (
                x_rh := rhmask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.so2_block_source.x_source_updated"
                    ][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_source_post_so2"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)

    display(Markdown(f"## Layer {layer_idx} x_target_updated"))
    print(
        {
            "rhescn": (
                x_rh := rhmask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.so2_block_target.x_target_updated"
                    ][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_target_post_so2"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)

    display(Markdown(f"## Layer {layer_idx} x_updated"))
    print(
        {
            "rhescn": (
                x_rh := rhmask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.x_updated"
                    ][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_updated"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1.0e-4)

    display(Markdown(f"## Layer {layer_idx} grid_act to_grid"))

    print(
        {
            "rhescn": (
                x_rh := (
                    rearrange(
                        rhescn.activations[
                            f"LayerBlock_{layer_idx}.MessageBlock.act_rotate_inv.x_grid"
                        ][0],
                        "E (res_beta res_alpha) C -> E res_beta res_alpha C",
                        res_beta=10,
                    )
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[
                        f"LayerBlock_{layer_idx}.message_block.grid_act.x_grid"
                    ][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)

    display(Markdown(f"## Layer {layer_idx} grid_act silu"))

    print(
        {
            "rhescn": (
                x_rh := (
                    rearrange(
                        rhescn.activations[
                            f"LayerBlock_{layer_idx}.MessageBlock.act_rotate_inv.x_grid_silu"
                        ][0],
                        "E (res_beta res_alpha) C -> E res_beta res_alpha C",
                        res_beta=10,
                    )
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[
                        f"LayerBlock_{layer_idx}.message_block.grid_act.x_grid_silu"
                    ][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)

    display(Markdown(f"## Layer {layer_idx} grid_act to_sphere"))

    print(
        {
            "rhescn": (
                x_rh := trimask_mmax(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.act_rotate_inv.x_sphere_silu"
                    ][0]
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[
                        f"LayerBlock_{layer_idx}.message_block.grid_act.x_sphere_silu"
                    ][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)

    display(Markdown(f"## Layer {layer_idx} rotate_inv"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.act_rotate_inv.x_sphere_rotated"
                    ][0]
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[
                        f"LayerBlock_{layer_idx}.message_block.x_target_rot_inv"
                    ][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)

    display(Markdown(f"## Layer {layer_idx} scatter"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.x_scattered"
                    ][0]
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[
                        f"LayerBlock_{layer_idx}.message_block.x_target_reduce"
                    ][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)

    display(Markdown(f"## Layer {layer_idx} x_message"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[f"LayerBlock_{layer_idx}.x_message"][0]
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[
                        f"LayerBlock_{layer_idx}.message_block.x_target_reduce"
                    ][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)

    display(Markdown(f"## Layer {layer_idx} x_message after SO2"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[f"LayerBlock_{layer_idx}.x_message"][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[f"LayerBlock_{layer_idx}.x_message"][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)

    display(Markdown(f"## Layer {layer_idx} conv: to_grid_mat"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.pointwise_grid_conv.to_grid_sh_tri"
                    ][0]
                )
            ),
            "escn": (
                x_escn := rearrange(
                    escn.activations[
                        f"LayerBlock_{layer_idx}.grid_conv.x_message_to_grid.to_grid_mat"
                    ][0],
                    "res_beta res_alpha l_sq -> (res_beta res_alpha) l_sq",
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)
    display(Markdown(f"## Layer {layer_idx} conv: x_grid"))

    print(
        {
            "rhescn": (
                x_rh := rhescn.activations[
                    f"LayerBlock_{layer_idx}.pointwise_grid_conv.s2_conv.x_grid"
                ][0]
            ),
            "escn": (
                x_escn := rearrange(
                    escn.activations[f"LayerBlock_{layer_idx}.grid_conv.x_grid"][0],
                    "N res_beta res_alpha C -> N (res_beta res_alpha) C",
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)

    display(Markdown(f"## Layer {layer_idx} conv: x_grid_message"))

    print(
        {
            "rhescn": (
                x_rh := rhescn.activations[
                    f"LayerBlock_{layer_idx}.pointwise_grid_conv.s2_conv.x_message_grid"
                ][0]
            ),
            "escn": (
                x_escn := rearrange(
                    escn.activations[
                        f"LayerBlock_{layer_idx}.grid_conv.x_grid_message"
                    ][0],
                    "N res_beta res_alpha C -> N (res_beta res_alpha) C",
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)

    display(Markdown(f"## Layer {layer_idx} conv: x_grid_conv"))

    print(
        {
            "rhescn": (
                x_rh := rhescn.activations[
                    f"LayerBlock_{layer_idx}.pointwise_grid_conv.s2_conv.x_grid_conv"
                ][0]
            ),
            "escn": (
                x_escn := rearrange(
                    escn.activations[f"LayerBlock_{layer_idx}.grid_conv.x_grid_conv"][
                        0
                    ],
                    "N res_beta res_alpha C -> N (res_beta res_alpha) C",
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)

    display(Markdown(f"## Layer {layer_idx} conv: x_message_from_grid"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.pointwise_grid_conv.x_message_updated"
                    ][0]
                )
            ),
            "escn": (
                x_escn := escn.activations[
                    f"LayerBlock_{layer_idx}.grid_conv.x_message_from_grid"
                ][0]
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)

    display(Markdown(f"## Layer {layer_idx} conv"))

    print(
        {
            "rhescn": (
                x_rh := trimask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.pointwise_grid_conv.x_message_updated"
                    ][0]
                )
            ),
            "escn": (
                x_escn := (
                    escn.activations[f"LayerBlock_{layer_idx}.x_message_updated"][0]
                )
            ),
        },
    )
    np.testing.assert_allclose(x_rh, x_escn, atol=1e-4)


# check_layer(0)

In [30]:
for i in range(12):
    check_layer(i)

## Layer 0 x_edge

{
    'rhescn': array[3137, 128] f32 n=401536 (1.5Mb) x∈[-0.278, 4.574] μ=-0.009 σ=0.064,
    'escn': array[3137, 128] f32 n=401536 (1.5Mb) x∈[-0.278, 4.574] μ=-0.009 σ=0.064
}

## Layer 0 x_source

{
    'rhescn': array[3137, 25, 128] f32 n=10038400 (38Mb) x∈[-0.887, 1.319] μ=0.002 σ=0.071,
    'escn': array[3137, 25, 128] f32 n=10038400 (38Mb) x∈[-0.887, 1.319] μ=0.002 σ=0.071
}

## Layer 0 x_target

...

{
    'rhescn': array[149, 110, 128] f32 n=2097920 (8.0Mb) x∈[-26.844, 21.829] μ=0.065 σ=1.146,
    'escn': array[149, 110, 128] f32 n=2097920 (8.0Mb) x∈[-26.844, 21.829] μ=0.065 σ=1.146
}

## Layer 11 conv: x_message_from_grid

{
    'rhescn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-29.861, 25.351] μ=0.010 σ=0.807,
    'escn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-29.861, 25.351] μ=0.010 σ=0.807
}

## Layer 11 conv

{
    'rhescn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-29.861, 25.351] μ=0.010 σ=0.807,
    'escn': array[149, 25, 128] f32 n=476800 (1.8Mb) x∈[-29.861, 25.351] μ=0.010 σ=0.807
}

In [ ]:
layer_idx = 1
display(Markdown(f"## Layer {layer_idx} rotate_inv"))


L = 4


def _select(x):
    range_ = slice((L) ** 2, (L + 1) ** 2)
    return x[:, range_]


print(
    {
        "rhescn": (
            x_rh := _select(
                trimask(
                    rhescn.activations[
                        f"LayerBlock_{layer_idx}.MessageBlock.act_rotate_inv.x_sphere_rotated"
                    ][0]
                )
            )
        ),
        "escn": (
            x_escn := _select(
                escn.activations[
                    f"LayerBlock_{layer_idx}.message_block.x_target_rot_inv"
                ][0]
            )
        ),
    },
)
np.testing.assert_allclose(x_rh, x_escn, atol=1e-5)

## Layer 1 rotate_inv

{
    'rhescn': array[3137, 9, 128] f32 n=3613824 (14Mb) x∈[-5.475, 4.458] μ=0.000 σ=0.097,
    'escn': array[3137, 9, 128] f32 n=3613824 (14Mb) x∈[-5.475, 4.458] μ=0.000 σ=0.097
}